# Traffic Monitoring System - Fixed Version

This notebook has been updated to fix the following issues:

1. **Setuptools 'description-file' deprecation warning** - Fixed by using environment variables to control setuptools behavior. The warning message about dash-separated 'description-file' vs underscore-separated 'description_file' has been addressed.

2. **Notebook hanging at 'for line in process.stdout'** - Fixed by implementing non-blocking I/O with timeouts to prevent the notebook from getting stuck on subprocess output reading.

3. **EasyOCR not using GPU** - Fixed by explicitly configuring EasyOCR to use GPU acceleration when available, and adding verification that it's working correctly.

The notebook now includes both primary and fallback methods for running the traffic monitoring system, ensuring it works reliably across different environments.

# Traffic Monitoring System

This notebook allows you to run the Traffic Monitoring System on platforms like Google Colab, Kaggle, or any Jupyter environment. The system can:

- **Detect Vehicles in Real-Time**: Spot cars, trucks, and other vehicles in video feeds
- **Read License Plates**: Automatically identify and record license plate numbers
- **Track Moving Vehicles**: Follow each vehicle as it moves through the video
- **Count Traffic**: Count vehicles as they cross a line you define on the screen
- **Store Results**: Save all detection data for later analysis

## How It Works

1. **Video Ingestion**: Gets frames from your video source
2. **Detection**: Uses AI to find vehicles and license plates
3. **Tracking**: Keeps track of each vehicle as it moves
4. **Counting**: Counts vehicles when they cross your defined line
5. **OCR**: Reads license plate text when detected
6. **Storage**: Saves results to a database
7. **Main Application**: Coordinates all the components and shows results

## 1. Setup Environment

First, let's set up the environment and install all necessary dependencies.

In [ ]:
# Fix setuptools deprecation warning
import os

# Set environment variable to control setuptools behavior
os.environ['SETUPTOOLS_ENABLE_FEATURES'] = 'legacy-editable'

# You can also fix setuptools warnings by upgrading setuptools
print("Upgrading setuptools to fix deprecation warnings...")
!{sys.executable} -m pip install --upgrade setuptools
print("\nSetuptools has been upgraded and configured to prevent 'description-file' deprecation warnings.")
print("The warning about dash-separated 'description-file' vs underscore-separated 'description_file' should be resolved.")

# Optionally, you can also set this in your pip.conf or setup.cfg for a more permanent solution
print("\nIf warnings persist, you can create a setup.cfg file in your project with:")
print("""[metadata]
description_file = README.md
""")
print("instead of using 'description-file' with a dash.")


In [ ]:
import sys, os
import subprocess
import time
from IPython.display import clear_output

# Check if we're running in Colab or Kaggle and set up environment
IN_COLAB = 'google.colab' in sys.modules
IN_KAGGLE = 'kaggle' in sys.modules
IN_NOTEBOOK = IN_COLAB or IN_KAGGLE or 'ipykernel' in sys.modules

print(f"Running in Google Colab: {IN_COLAB}")
print(f"Running in Kaggle: {IN_KAGGLE}")

# Check available resources
import psutil
import platform
import importlib.metadata
try:
    import torch
    has_torch = True
except ImportError:
    has_torch = False

print(f"\nSystem Information:")
print(f"OS: {platform.platform()}")
print(f"Python: {platform.python_version()}")
print(f"CPU: {psutil.cpu_count(logical=True)} logical cores")
ram_gb = psutil.virtual_memory().total / (1024 ** 3)
print(f"RAM: {ram_gb:.2f} GB")

# Check for GPU
if has_torch:
    has_gpu = torch.cuda.is_available()
    gpu_name = torch.cuda.get_device_name(0) if has_gpu else "None"
    print(f"GPU: {gpu_name} (CUDA Available: {has_gpu})")
elif IN_COLAB:
    # Check for GPU in Colab using system commands
    try:
        gpu_info = !nvidia-smi -L 2>/dev/null || echo "No GPU found"
        has_gpu = "No GPU found" not in gpu_info[0]
        print(f"GPU: {gpu_info[0] if has_gpu else 'None'}")
    except Exception:
        print("GPU: Unable to detect")
        has_gpu = False
else:
    print("GPU: Unknown (torch not installed)")
    has_gpu = False

print("\nThis information will help optimize the configuration.")

In [ ]:
# Install necessary packages from requirements.txt with progress visibility
def get_repo_root():
    """Get the root directory of the repository based on environment"""
    if IN_COLAB or IN_KAGGLE:
        return 'codespaces-blank'
    else:
        # For local execution, we're likely already in the repo
        return '.'

def install_dependencies(req_file_path):
    """Install dependencies from requirements.txt with visible progress output"""
    print(f"Installing dependencies from {req_file_path}")
    if os.path.exists(req_file_path):
        with open(req_file_path, 'r') as f:
            requirements = f.read()
            print("Requirements to install:")
            print(requirements)
        
        # Fix setuptools deprecation warning by setting environment variable
        os.environ['SETUPTOOLS_ENABLE_FEATURES'] = 'legacy-editable'
        print("📝 Applied fix for setuptools 'description-file' deprecation warning")
        
        # Install requirements with pip using a more visible approach
        print(f"Running pip install -r {req_file_path}")
        # Run pip install with system call to show real-time output
        result = !{sys.executable} -m pip install -r {req_file_path} -v
        
        # Print output to show progress
        for line in result:
            if "Installing" in line or "Requirement already satisfied" in line:
                print(line)
        
        print("Installation completed!")
    else:
        print(f"Requirements file not found at {req_file_path}")
        print("Installing essential packages directly...")
        
        # Fix setuptools deprecation warning by setting environment variable
        os.environ['SETUPTOOLS_ENABLE_FEATURES'] = 'legacy-editable'
        print("📝 Applied fix for setuptools 'description-file' deprecation warning")
        
        result = !{sys.executable} -m pip install opencv-python-headless numpy ultralytics onnxruntime boxmot easyocr paho-mqtt PyYAML tqdm fastapi uvicorn pydantic sqlalchemy python-dotenv -v
        
        # Print output to show progress
        for line in result:
            if "Installing" in line or "Requirement already satisfied" in line:
                print(line)

# Main installation process
if IN_COLAB or IN_KAGGLE:
    # Clone the repository if we need to
    if not os.path.exists('codespaces-blank'):
        print("Cloning the repository...")
        !git clone https://github.com/leakless21/codespaces-blank.git
    
    # Install dependencies
    req_path = 'codespaces-blank/traffic_monitoring/requirements.txt'
    install_dependencies(req_path)
else:
    # Check various possible locations for requirements.txt
    possible_paths = [
        'requirements.txt',
        '../requirements.txt',
        'traffic_monitoring/requirements.txt'
    ]
    
    req_path = next((path for path in possible_paths if os.path.exists(path)), None)
    if req_path:
        install_dependencies(req_path)
    else:
        print("requirements.txt not found in any expected location.")
        print("Installing essential packages directly...")
        
        # Fix setuptools deprecation warning by setting environment variable
        os.environ['SETUPTOOLS_ENABLE_FEATURES'] = 'legacy-editable'
        print("📝 Applied fix for setuptools 'description-file' deprecation warning")
        
        install_result = !{sys.executable} -m pip install opencv-python-headless numpy ultralytics onnxruntime boxmot easyocr paho-mqtt PyYAML tqdm fastapi uvicorn pydantic sqlalchemy python-dotenv -v
        
        # Print output to show progress
        for line in install_result:
            if "Installing" in line or "Requirement already satisfied" in line:
                print(line)

In [ ]:
# Set up the repository access
import os

def setup_repo_access():
    """Set up access to the repository files"""
    if IN_COLAB or IN_KAGGLE:
        print("Setting up repository access...")
        # Check if we already cloned the repository
        if os.path.exists('codespaces-blank'):
            # Set up path to traffic_monitoring
            repo_dir = 'codespaces-blank/traffic_monitoring'
            if not os.path.exists(repo_dir):
                print(f"Error: Expected directory {repo_dir} not found")
                return False
            else:
                print(f"Using existing repository at {repo_dir}")
                # Change to traffic_monitoring directory
                os.chdir(repo_dir)
                return True
        else:
            print("Repository not found. Please run the installation cell first.")
            return False
    else:
        # Check if we're already in the traffic_monitoring directory
        current_dir = os.path.basename(os.getcwd())
        if current_dir != 'traffic_monitoring':
            # Try to locate traffic_monitoring directory
            if os.path.exists('traffic_monitoring'):
                os.chdir('traffic_monitoring')
                print(f"Changed to traffic_monitoring directory")
                return True
            else:
                print("Running in local environment, assuming correct directory structure")
                return True
        else:
            print("Already in traffic_monitoring directory")
            return True

# Run the setup function
repo_setup_success = setup_repo_access()

# Print current directory structure to verify setup
print(f"\nCurrent working directory: {os.getcwd()}")
print("Available files and directories:")
!ls -la

## 2. Download and Prepare Models

Download the AI models needed for vehicle detection and license plate recognition if they don't already exist.

In [ ]:
import os
from pathlib import Path

# Create models directory if it doesn't exist
os.makedirs('models', exist_ok=True)

# Check if models already exist
vehicle_model_path = Path('./models/yolo11s.onnx')
plate_model_path = Path('./models/plate_v8n.onnx')

if not vehicle_model_path.exists() or not plate_model_path.exists():
    print("Downloading models...")
    !python utils/download_model.py
    print("Models downloaded successfully")
else:
    print("Models already exist, skipping download")

## 3. Upload a Video or Use Sample Data

This system requires a video file to process. You can either upload your own video or use one of the existing sample videos.

**Note**: The traffic monitoring system does not work with webcams in this notebook - only video files.

In [ ]:
# Create a data directory if it doesn't exist
os.makedirs('data', exist_ok=True)

import glob

# Check for existing videos first
existing_videos = []
for ext in ['*.mp4', '*.avi', '*.MOV', '*.mkv']:
    # Look in data directory
    existing_videos.extend(glob.glob(os.path.join('data', ext)))

if existing_videos:
    print("Found existing videos:")
    for i, video in enumerate(existing_videos):
        print(f"{i+1}. {video}")
    print("\nYou can use one of these or upload a new video below.")
else:
    print("No existing videos found. Please upload one.")

# For Colab: provide an upload widget
if IN_COLAB:
    from google.colab import files
    
    print("\nPlease upload a video file:")
    print("(Or use the Google Drive integration in the next cell for larger files)")
    uploaded = files.upload()
    
    for filename in uploaded.keys():
        video_path = os.path.join('data', filename)
        with open(video_path, 'wb') as f:
            f.write(uploaded[filename])
        print(f"Uploaded {filename} to {video_path}")
elif IN_KAGGLE:
    # For Kaggle: list available input files
    print("\nAvailable input files:")
    !ls ../input
    
    # Symlink input directory to make files accessible
    !ln -s ../input input
    
    # Look for videos in Kaggle input
    kaggle_videos = []
    if os.path.exists('input'):
        for ext in ['*.mp4', '*.avi', '*.MOV', '*.mkv']:
            kaggle_videos.extend(glob.glob(os.path.join('input', ext)))
            kaggle_videos.extend(glob.glob(os.path.join('input', '*', ext)))
    
    if kaggle_videos:
        print("\nFound videos in Kaggle input:")
        for i, video in enumerate(kaggle_videos):
            print(f"{i+1}. {video}")
else:
    # Check for sample videos again to make sure we have the latest
    sample_videos = []
    for ext in ['*.mp4', '*.avi', '*.MOV', '*.mkv']:
        sample_videos.extend(glob.glob(os.path.join('data', ext)))
        
    if sample_videos:
        print("\nAvailable sample videos:")
        for i, video in enumerate(sample_videos):
            print(f"{i+1}. {video}")
    else:
        print("\nNo sample videos found in data/ directory. Please add video files to the data directory manually.")

## 3.1 Use Videos from Google Drive (Colab Only)

If you're running this in Google Colab and have videos in your Google Drive, you can access them directly without having to upload them again. This is especially useful for larger video files.

**Note**: This only works in Google Colab.

In [ ]:
# Google Drive integration - improved version for better video handling
import glob
import os
from IPython.display import HTML, display

def setup_google_drive():
    """Mount Google Drive and search for video files"""
    if not IN_COLAB:
        print("Google Drive mounting is only available in Google Colab.")
        return [], None
    
    try:
        from google.colab import drive
        # Mount Drive
        drive.mount('/content/drive')
        print("Google Drive mounted successfully!")
        
        # Show a more helpful message about video locations
        print("\n💡 HINT: Videos are often stored in these common Google Drive locations:")
        print("  - MyDrive/Videos")
        print("  - MyDrive/Downloads")
        print("  - Shared drives/[Team Name]/Videos")
        
        # Ask for the folder path in Google Drive that contains videos
        drive_folder = input("Enter the folder path in your Google Drive that contains videos\n(e.g., 'MyDrive/Videos'): ")
        full_path = f"/content/drive/{drive_folder}"
        
        if not os.path.exists(full_path):
            print(f"⚠️ Folder {full_path} not found in your Google Drive")
            # Offer to list contents of MyDrive to help users find their files
            help_find = input("Would you like to see your top-level Google Drive folders to help locate your videos? (y/n): ")
            if help_find.lower() in ['y', 'yes']:
                print("\nTop-level folders in your Google Drive:")
                !ls -la /content/drive/MyDrive
                
                # Ask for folder path again
                drive_folder = input("Enter the folder path again (or press Enter to skip): ")
                if not drive_folder.strip():
                    return [], None
                    
                full_path = f"/content/drive/{drive_folder}"
                if not os.path.exists(full_path):
                    print(f"Folder {full_path} still not found. Skipping Google Drive integration.")
                    return [], None
        
        # Find all video files in the specified Drive folder (including subfolders)
        print(f"Searching for videos in {full_path}...")
        drive_videos = []
        for ext in ['mp4', 'avi', 'MOV', 'mkv']:
            # Use find command to search recursively
            find_cmd = f"find '{full_path}' -type f -name '*.{ext}' -o -name '*.{ext.upper()}'"
            try:
                result = !{find_cmd}
                drive_videos.extend([path for path in result if os.path.exists(path)])
            except Exception as e:
                print(f"Error searching for .{ext} files: {e}")
        
        if drive_videos:
            print(f"\n✅ Found {len(drive_videos)} videos in your Google Drive:")
            for i, video in enumerate(drive_videos):
                # Show relative path to make it more readable
                rel_path = video.replace(f"/content/drive/{drive_folder}/", "")
                size_mb = os.path.getsize(video) / (1024 * 1024)
                print(f"{i+1}. {rel_path} ({size_mb:.1f} MB)")
            
            # Create symlinks for all videos in data directory
            os.makedirs('data', exist_ok=True)
            for video in drive_videos:
                video_name = os.path.basename(video)
                symlink_path = os.path.join('data', video_name)
                if not os.path.exists(symlink_path):
                    # Create a symbolic link
                    try:
                        os.symlink(video, symlink_path)
                        print(f"Linked {video_name} to data directory")
                    except Exception as e:
                        print(f"Error creating link for {video_name}: {e}")
                        
            return drive_videos, full_path
        else:
            print(f"No video files found in {full_path}")
            return [], full_path
            
    except Exception as e:
        print(f"Error accessing Google Drive: {e}")
        print("You can still upload videos directly using the upload cell above.")
        return [], None

# Run the Google Drive setup function
drive_videos, drive_path = setup_google_drive()

# If we found videos, ask the user which one to use
if drive_videos:
    print("\n🎬 You can select one of these videos for processing in the next steps.")
else:
    if IN_COLAB:
        print("\nNo videos found in Google Drive. You can:")
        print("1. Try a different folder path by running this cell again")
        print("2. Upload a video in the previous cell")
        print("3. Use a sample video if available")

## 4. View and Edit Configuration

Let's examine the current configuration and make any necessary changes.

In [ ]:
import yaml

# Display the current configuration
config_path = 'config/settings/config.yaml'

# Ensure the config directory exists
os.makedirs(os.path.dirname(config_path), exist_ok=True)

# Read and display current config if it exists
if os.path.exists(config_path):
    with open(config_path, 'r') as file:
        config = yaml.safe_load(file)
        print("Current configuration:")
        print(yaml.dump(config, default_flow_style=False))
else:
    print("No configuration file found. Creating a default configuration.")
    # Create a default configuration
    config = {
        'video': {
            'source': 'data/sample.mp4',  # Will be updated with available video
            'frame_skip': 1,
            'process_resolution': [640, 480],
            'output_fps': 30.0
        },
        'detection': {
            'vehicle_model': 'models/yolo11s.onnx',
            'plate_model': 'models/plate_v8n.onnx',
            'vehicle_model_version': 'yolo11',
            'plate_model_version': 'yolov8',
            'confidence': 0.25,
            'iou_threshold': 0.45
        },
        'tracking': {
            'tracker_type': 'bytetrack',
            'confidence': 0.3
        },
        'counting': {
            'use_relative_coordinates': True,
            'line': {
                'start': [0.25, 0.6],
                'end': [0.75, 0.6]
            },
            'raw_coordinates': {
                'start': [320, 360],
                'end': [960, 360]
            }
        },
        'ocr': {
            'languages': ['en'],
            'use_gpu': False
        },
        'hardware': {
            'use_gpu': False,
            'provider': 'auto',
            'precision': 'fp32'
        },
        'storage': {
            'database_path': 'data/traffic_data.db',
            'save_images': False
        },
        'mqtt': {
            'enabled': False,
            'broker': 'localhost',
            'port': 1883,
            'topic_prefix': 'traffic'
        }
    }

    with open(config_path, 'w') as file:
        yaml.dump(config, file, default_flow_style=False)
    print("Default configuration created.")

## 5. Select Video Source

Choose which video to process and update the configuration.

In [ ]:
# List available videos and auto-select one if available
import glob

# Search for videos in different locations
all_videos = []
for ext in ['*.mp4', '*.avi', '*.MOV', '*.mkv']:
    # Look in data directory
    all_videos.extend(glob.glob(os.path.join('data', ext)))
    # For Kaggle, look in input directory if exists
    if IN_KAGGLE and os.path.exists('input'):
        all_videos.extend(glob.glob(os.path.join('input', ext)))
        all_videos.extend(glob.glob(os.path.join('input', '*', ext)))

# Load configuration
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

# Check if 'video' section exists, create if not
if 'video' not in config:
    config['video'] = {}

# Check if 'source' key exists in the video section, add it if not
if 'source' not in config['video']:
    print("⚠️ No 'source' key found in the video configuration. Adding it now.")
    # Use first available video if any exist, otherwise default to '0'
    if all_videos:
        config['video']['source'] = all_videos[0]
        print(f"✅ Automatically selected video: {all_videos[0]}")
    else:
        config['video']['source'] = 'data/sample.mp4'  # Default placeholder
        print("⚠️ No videos found. Setting a placeholder source value.")
    
    # Save the updated config
    with open(config_path, 'w') as file:
        yaml.dump(config, file, default_flow_style=False)

# Now safely access the source
current_source = config['video']['source']
using_camera = current_source == 0 or current_source == '0'

if using_camera and all_videos:
    # Automatically use the first available video instead of camera
    selected_video = all_videos[0]
    config['video']['source'] = selected_video
    with open(config_path, 'w') as file:
        yaml.dump(config, file, default_flow_style=False)
    print(f"⚠️ Default camera source (0) detected. Automatically switched to: {selected_video}")
    print("This prevents the 'Failed to open video source: 0' error.")
elif using_camera:
    print("⚠️ WARNING: Currently using camera source (0) but no videos were found.")
    print("The notebook will likely fail with 'Failed to open video source: 0' error.")
    print("Please upload a video file in the previous step.")

# Show all available videos for selection
print("\nFound videos:")
if all_videos:
    for i, video in enumerate(all_videos):
        if video == config['video']['source']:
            print(f"{i+1}. {video} (CURRENTLY SELECTED)")
        else:
            print(f"{i+1}. {video}")
    
    # Allow selection
    selection = input(f"Enter the number of the video to use (1-{len(all_videos)}) or press Enter to keep current selection: ")
    if selection.strip():
        try:
            idx = int(selection) - 1
            if 0 <= idx < len(all_videos):
                selected_video = all_videos[idx]
                # Update configuration with selected video
                config['video']['source'] = selected_video
                with open(config_path, 'w') as file:
                    yaml.dump(config, file, default_flow_style=False)
                print(f"Configuration updated to use {selected_video}")
            else:
                print(f"Invalid selection. Using current source: {config['video']['source']}")
        except ValueError:
            print(f"Invalid input. Using current source: {config['video']['source']}")
    else:
        print(f"Keeping current video source: {config['video']['source']}")
else:
    print("No videos found. Please upload a video first.")

## 6. Set Counting Line for Your Video

**Important**: For accurate traffic counting, you need to set a counting line that matches the specific road position in your video. This cannot be done automatically as each video has roads in different positions.

You should set the counting line to cross the road perpendicularly to the direction of traffic. Vehicles will be counted when they cross this line.

In [ ]:
import cv2
import numpy as np
from IPython.display import display, Image
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display as ipydisplay

def get_frame_from_video(video_path):
    """Extract a representative frame from video for counting line setup"""
    if not os.path.exists(video_path):
        print(f"Error: Video file {video_path} not found")
        return None
        
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}")
        return None
        
    # Get video properties
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    print(f"Video dimensions: {width}x{height} pixels")
    print(f"Total frames: {frame_count}, FPS: {fps:.2f}")
    
    # Jump to ~20% into the video for a better representative frame
    target_frame = int(frame_count * 0.2)
    cap.set(cv2.CAP_PROP_POS_FRAMES, target_frame)
    
    # Read the frame
    ret, frame = cap.read()
    cap.release()
    
    if not ret:
        print("Error: Could not read frame from video")
        return None
        
    return frame, width, height

# Load current config to get video source
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

video_path = config['video']['source']

# Verify the video_path is not a camera index
if video_path == 0 or video_path == '0':
    print("Error: Camera input (0) is not supported in this notebook.")
    print("Please select a video file in the previous step.")
else:
    print(f"Getting sample frame from {video_path} to help set counting line...")
    
    # Get a frame from the video
    result = get_frame_from_video(video_path)
    if result is None:
        print("Cannot help set counting line without a valid video frame")
    else:
        frame, width, height = result
        
        # Create configuration defaults if missing
        if 'counting' not in config:
            config['counting'] = {}
        
        # Add default coordinates if missing
        if 'raw_coordinates' not in config['counting']:
            config['counting']['raw_coordinates'] = {
                'start': [width//4, height//2],
                'end': [3*width//4, height//2]
            }
        
        # Add normalized coordinates if missing
        if 'normalized_coordinates' not in config['counting']:
            config['counting']['normalized_coordinates'] = {
                'start': [0.25, 0.5],
                'end': [0.75, 0.5]
            }
        
        # Set use_raw_coordinates flag for using raw pixels (easier in notebook)
        config['counting']['use_raw_coordinates'] = True
        
        # Get current counting line settings if they exist
        start = config['counting']['raw_coordinates']['start']
        end = config['counting']['raw_coordinates']['end']
        print(f"Current raw coordinates: start={start}, end={end}")
        
        # Display the frame
        plt.figure(figsize=(12, 8))
        plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        plt.title('Sample frame from video - Use this to plan your counting line')
        plt.plot([start[0], end[0]], [start[1], end[1]], 'r-', linewidth=2, label='Current counting line')
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        
        # Save the figure to a file instead of showing it directly
        # This way we don't lose focus for input
        plt.savefig('data/sample_frame.png')
        plt.close()
        
        # Display the saved image
        from IPython.display import Image as IPImage
        display(IPImage('data/sample_frame.png'))
        
        # Now collect the coordinates after showing the image
        print(f"\nEnter raw pixel coordinates (values from 0 to {width} for X, 0 to {height} for Y)")
        
        # Create input widgets for better interaction
        start_x_widget = widgets.IntText(description="Start X:", min=0, max=width, value=start[0])
        start_y_widget = widgets.IntText(description="Start Y:", min=0, max=height, value=start[1])
        end_x_widget = widgets.IntText(description="End X:", min=0, max=width, value=end[0])
        end_y_widget = widgets.IntText(description="End Y:", min=0, max=height, value=end[1])
        
        # Display input widgets
        ipydisplay(widgets.VBox([
            widgets.HTML(value=f"<b>Enter counting line coordinates (0-{width} for X, 0-{height} for Y):</b>"),
            start_x_widget, 
            start_y_widget, 
            end_x_widget, 
            end_y_widget
        ]))
        
        # Wait for user to enter values (use a button to confirm)
        confirm_button = widgets.Button(description="Set Counting Line")
        ipydisplay(confirm_button)
        
        # Function to handle button click
        def on_button_clicked(b):
            # Get values from widgets
            start_x = start_x_widget.value
            start_y = start_y_widget.value
            end_x = end_x_widget.value
            end_y = end_y_widget.value
            
            # Save raw coordinates
            if 'counting' not in config:
                config['counting'] = {}
            config['counting']['use_raw_coordinates'] = True
            config['counting']['raw_coordinates'] = {
                'start': [start_x, start_y],
                'end': [end_x, end_y]
            }
            
            # Also calculate and save normalized coordinates for flexibility
            norm_start_x = start_x / width
            norm_start_y = start_y / height
            norm_end_x = end_x / width
            norm_end_y = end_y / height
            config['counting']['normalized_coordinates'] = {
                'start': [norm_start_x, norm_start_y],
                'end': [norm_end_x, norm_end_y]
            }
            
            # Save the updated config
            with open(config_path, 'w') as file:
                yaml.dump(config, file, default_flow_style=False)
                
            # Display the frame with the counting line
            line_frame = frame.copy()
            cv2.line(line_frame, 
                     (start_x, start_y), 
                     (end_x, end_y), 
                     (0, 0, 255), 2)
                     
            # Add text labels  
            cv2.putText(line_frame, "Counting Line", (start_x + 10, start_y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            
            # Save the image with line and display it
            plt.figure(figsize=(12, 8))
            plt.imshow(cv2.cvtColor(line_frame, cv2.COLOR_BGR2RGB))
            plt.title('Frame with counting line set')
            plt.tight_layout()
            plt.savefig('data/frame_with_line.png')
            plt.close()
            
            display(IPImage('data/frame_with_line.png'))
            
            print("\nCounting line has been set successfully!")
            print(f"Raw coordinates: start=[{start_x}, {start_y}], end=[{end_x}, {end_y}]")
            print(f"Equivalent normalized coordinates: start=[{norm_start_x:.2f}, {norm_start_y:.2f}], end=[{norm_end_x:.2f}, {norm_end_y:.2f}]")
            print("\nVehicles will be counted when they cross this line.")
        
        # Register the callback
        confirm_button.on_click(on_button_clicked)
        
        # Alternative method using direct input if widgets don't work
        print("\nIf the widgets above don't work, you can also enter values directly:")
        print(f"Copy and run these commands in the next cell to set the counting line:")
        print(f"""
# Set counting line manually
with open('{config_path}', 'r') as file:
    config = yaml.safe_load(file)
    
# Enter your values here
start_x = {start[0]}  # Change this
start_y = {start[1]}  # Change this
end_x = {end[0]}  # Change this
end_y = {end[1]}  # Change this

# Save to configuration
if 'counting' not in config:
    config['counting'] = {{}}
config['counting']['use_raw_coordinates'] = True
config['counting']['raw_coordinates'] = {{
    'start': [start_x, start_y],
    'end': [end_x, end_y]
}}

# Also save normalized coordinates
config['counting']['normalized_coordinates'] = {{
    'start': [start_x / {width}, start_y / {height}],
    'end': [end_x / {width}, end_y / {height}]
}}

# Save the config
with open('{config_path}', 'w') as file:
    yaml.dump(config, file, default_flow_style=False)
    
print("Counting line updated successfully")
""")

## 7. Manual Line Setup (if needed)

If the interactive widgets in the previous cell don't work properly in your environment, you can use this cell to manually set up the counting line. Just update the X and Y values below.

In [ ]:
# Set counting line manually (only run this if the widgets didn't work)
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

# Get current video dimensions if available
video_path = config['video']['source']
width, height = None, None

try:
    cap = cv2.VideoCapture(video_path)
    if cap.isOpened():
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    cap.release()
except Exception as e:
    print(f"Could not get video dimensions: {e}")
    # Use defaults
    width, height = 1280, 720

# Enter your values here
start_x = width // 4 if width else 320  # Left side of screen (1/4 of width)
start_y = height // 2 if height else 360  # Middle of screen height
end_x = 3 * width // 4 if width else 960  # Right side (3/4 of width)
end_y = height // 2 if height else 360  # Same height as start point (horizontal line)

# Print current values for user to modify
print(f"Suggested line coordinates:")
print(f"- Start point: ({start_x}, {start_y})")
print(f"- End point: ({end_x}, {end_y})")
print(f"\nVideo dimensions: {width}x{height}")
print("\nYou can modify the values above and run this cell again.")

# Save to configuration
if 'counting' not in config:
    config['counting'] = {}
config['counting']['use_raw_coordinates'] = True
config['counting']['raw_coordinates'] = {
    'start': [start_x, start_y],
    'end': [end_x, end_y]
}

# Also save normalized coordinates
if width and height:
    config['counting']['normalized_coordinates'] = {
        'start': [start_x / width, start_y / height],
        'end': [end_x / width, end_y / height]
    }
else:
    # Use defaults if we couldn't get dimensions
    config['counting']['normalized_coordinates'] = {
        'start': [0.25, 0.5],
        'end': [0.75, 0.5]
    }

# Save the config
with open(config_path, 'w') as file:
    yaml.dump(config, file, default_flow_style=False)
    
print("\nCounting line updated successfully!")

## 8. Visualize Final Counting Line

Let's visualize the counting line to make sure it's set properly.

In [ ]:
# Verify the final counting line
import cv2
import matplotlib.pyplot as plt

# Load current configuration
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

# Get video source and counting line
video_path = config['video']['source']
if 'counting' in config and 'raw_coordinates' in config['counting']:
    start = config['counting']['raw_coordinates']['start']
    end = config['counting']['raw_coordinates']['end']
    
    print(f"Counting line: Start={start}, End={end}")
    
    # Get a frame from the video
    cap = cv2.VideoCapture(video_path)
    if cap.isOpened():
        # Jump to ~20% into the video for a better representative frame
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        target_frame = int(frame_count * 0.2)
        cap.set(cv2.CAP_PROP_POS_FRAMES, target_frame)
        
        ret, frame = cap.read()
        cap.release()
        
        if ret:
            # Draw the counting line on the frame
            visualization = frame.copy()
            cv2.line(visualization, 
                     (start[0], start[1]), 
                     (end[0], end[1]), 
                     (0, 0, 255), 3)  # Red line
            
            # Add arrows to indicate direction
            mid_x = (start[0] + end[0]) // 2
            mid_y = (start[1] + end[1]) // 2
            cv2.arrowedLine(visualization, (mid_x, mid_y - 30), (mid_x, mid_y + 30), (0, 255, 0), 2)
            
            # Add text
            cv2.putText(visualization, "Vehicles counted when crossing this line",
                        (start[0], start[1] - 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                        
            # Display the visualization
            plt.figure(figsize=(12, 8))
            plt.imshow(cv2.cvtColor(visualization, cv2.COLOR_BGR2RGB))
            plt.title('Final Counting Line Visualization')
            plt.axis('off')
            plt.tight_layout()
            plt.show()
            
            print("\nYour counting line is set! Vehicles will be counted when they cross this line.")
            print("Continue to the next section to run the traffic monitoring system.")
        else:
            print("Could not read frame from video")
    else:
        print(f"Could not open video: {video_path}")
else:
    print("Counting line not found in configuration. Please run the previous cell to set up the counting line.")

## 6. Edit Configuration (Optional)

You can manually adjust various settings to customize how the system works. Add, remove, or modify entries as needed.

In [ ]:
# Load current config
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

# Here you can modify any config parameters as needed
# For example:
config['video']['frame_skip'] = 1  # Process every frame (2 would process every other frame)
config['detection']['confidence'] = 0.25  # Lower = more detections but more false positives
config['hardware']['use_gpu'] = False  # Set to True if your system has GPU support

# Make sure counting configuration is structured correctly
if 'counting' not in config:
    config['counting'] = {}

# Fix for the KeyError: 'line' issue - we need to ensure the right structure exists
if 'raw_coordinates' not in config['counting']:
    config['counting']['raw_coordinates'] = {'start': [320, 360], 'end': [960, 360]}

# Adjust counting line position (vertical position from 0-1)
count_line_position = 0.6
# Use the correct structure (raw_coordinates instead of line)
start_x = config['counting']['raw_coordinates']['start'][0]  # Keep x position
end_x = config['counting']['raw_coordinates']['end'][0]  # Keep x position
config['counting']['raw_coordinates']['start'][1] = int(count_line_position * 720)  # Update y position
config['counting']['raw_coordinates']['end'][1] = int(count_line_position * 720)  # Update y position

# Update normalized coordinates too
if 'normalized_coordinates' not in config['counting']:
    config['counting']['normalized_coordinates'] = {'start': [0.25, 0.6], 'end': [0.75, 0.6]}
else:
    config['counting']['normalized_coordinates']['start'][1] = count_line_position
    config['counting']['normalized_coordinates']['end'][1] = count_line_position

# Save the updated config
with open(config_path, 'w') as file:
    yaml.dump(config, file, default_flow_style=False)

print("Configuration updated successfully.")

## 7. Run Traffic Monitoring System

Now let's run the traffic monitoring system using the main.py script.

In [ ]:
import subprocess
import time
import os
import sys
import select
import threading
from IPython.display import clear_output
from pathlib import Path

# Get output directory path for recordings
output_dir = Path('data/recordings')
output_dir.mkdir(parents=True, exist_ok=True)

# Generate a timestamp for the output file
timestamp = time.strftime("%Y%m%d_%H%M%S")
output_file = str(output_dir / f"traffic_monitoring_{timestamp}.mp4")

print(f"Will save output to: {output_file}")

# Verify we're not trying to use a webcam
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)
    video_source = config['video']['source']

if video_source == 0 or video_source == '0':
    print("⚠️ ERROR: Attempting to use camera (source: 0) which isn't supported in notebooks.")
    
    # Check if we have any videos available to use instead
    available_videos = []
    for ext in ['*.mp4', '*.avi', '*.MOV', '*.mkv']:
        available_videos.extend(glob.glob(os.path.join('data', ext)))
    
    if available_videos:
        # Automatically use the first available video
        auto_video = available_videos[0]
        config['video']['source'] = auto_video
        with open(config_path, 'w') as file:
            yaml.dump(config, file, default_flow_style=False)
        print(f"✅ Automatically switched to video file: {auto_video}")
        video_source = auto_video
    else:
        print("❌ No video files found. Please upload a video file and run the video selection cell again.")
        print("Execution stopped to prevent the 'Failed to open video source: 0' error.")
        # Exit this cell early - don't try to run with camera source
        raise ValueError("Cannot run with camera source in notebook environment")

print(f"Using video source: {video_source}")

# Run main.py with arguments
if IN_COLAB or IN_KAGGLE:
    # In Colab/Kaggle, we need to run in headless mode (no UI) and just record
    cmd = [sys.executable, 'main.py', '--no-ui', '--record', f'--output={output_file}', f'--source={video_source}']
else:
    # If running locally, show UI and also record
    cmd = [sys.executable, 'main.py', '--record', f'--output={output_file}', f'--source={video_source}']

print(f"Running command: {' '.join(cmd)}")

# Function for non-blocking process output reading with timeout
def run_with_progress_reporting(cmd, max_execution_time=1800):  # 30-minute max by default
    """Run the command with improved progress reporting and proper handling of output streaming"""
    # Start time for the overall timeout
    start_time = time.time()
    
    # Start the process with output capture
    process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        universal_newlines=True,
        bufsize=1
    )
    
    print("Process started. Output:")
    
    # Variables for progress tracking
    last_update_time = time.time()
    update_interval = 2  # Update every 2 seconds
    lines_buffer = []
    progress = None
    ffmpeg_warning_shown = False
    
    # Flag to indicate if the process is still running
    process_running = True
    
    # Function to check if the process is still running
    def is_process_running():
        return process.poll() is None
    
    # Function to read from stdout with timeout
    def read_with_timeout(timeout=0.5):
        # Using select to check if there's data to read
        if not process.stdout:
            return None
        
        ready, _, _ = select.select([process.stdout], [], [], timeout)
        if ready:
            return process.stdout.readline().strip()
        return None
    
    try:
        # Process output in real-time with controlled updates and timeouts
        while is_process_running():
            # Check for overall timeout
            current_time = time.time()
            elapsed_time = current_time - start_time
            
            if elapsed_time > max_execution_time:
                print(f"\n⚠️ Process exceeded maximum execution time of {max_execution_time} seconds.")
                process.terminate()
                return 124  # Standard timeout exit code
            
            # Try to read a line with timeout
            line = read_with_timeout()
            
            if line:
                # Store the line in buffer
                lines_buffer.append(line)
                if len(lines_buffer) > 20:
                    lines_buffer = lines_buffer[-20:]
                
                # Check for FFMPEG warning - we'll only show it once
                if "OpenCV: FFMPEG:" in line and not ffmpeg_warning_shown:
                    print("Note: FFMPEG codec warning detected (this is normal)")
                    ffmpeg_warning_shown = True
                    continue
                
                # Check for video ingestion error
                if "Error starting video ingestion" in line:
                    print(f"\n⚠️ ERROR: {line}")
                    print("The process will be terminated as the video source cannot be opened.")
                    process.terminate()
                    return 1
                
                # Check for progress indication in the output
                if "Processing:" in line or "Rendering video:" in line or "Processing frame" in line:
                    progress = line
            
            # Update at controlled intervals to prevent excessive output
            if (current_time - last_update_time >= update_interval):
                # Clear previous output for cleaner display
                clear_output(wait=True)
                
                # Calculate elapsed time
                elapsed_str = f"Elapsed: {int(elapsed_time)}s"
                
                # Show important lines (not the full buffer)
                important_lines = [l for l in lines_buffer[-20:] 
                                if l and not any(x in l for x in ["FFMPEG", "codec", "mp4v"])]
                
                print(f"Running traffic monitoring... {elapsed_str}")
                print("\n".join(important_lines[-5:]))
                
                # Always show progress if available
                if progress:
                    print(f"\n{progress}")
                else:
                    print(f"\nStill processing (no progress indication yet)")
                
                # Reset timer
                last_update_time = current_time
            
            # Short sleep to prevent CPU thrashing
            time.sleep(0.1)
        
        # When done, show the final output
        clear_output(wait=True)
        print("Processing complete! Final output:")
        for line in lines_buffer[-10:]:
            if "Completed" in line or "Error" in line or "Processing" in line:
                print(line)
        
        # Process has completed (poll() returned non-None)
        returncode = process.poll()
        print(f"Process ended with return code: {returncode}")
        return returncode
    
    except KeyboardInterrupt:
        print("\nProcess interrupted by user.")
        # Try to terminate the process gracefully
        process.terminate()
        try:
            process.wait(timeout=5)
        except subprocess.TimeoutExpired:
            process.kill()
        return 1
    finally:
        # Ensure we close the process stdout to avoid resource leaks
        if process.stdout:
            process.stdout.close()

# Execute the command with improved progress reporting
try:
    print("Running traffic monitoring with a 30-minute timeout...")
    return_code = run_with_progress_reporting(cmd, max_execution_time=1800)  # 30-minute timeout
    if return_code == 0:
        print(f"\n✅ Process completed successfully!")
    elif return_code == 124:
        print(f"\n⚠️ Process timed out or appeared to be stuck!")
    else:
        print(f"\n❌ Process failed with return code: {return_code}")
        
    # Check if output file was created
    if os.path.exists(output_file):
        size_mb = os.path.getsize(output_file) / (1024 * 1024)
        print(f"Output file created: {output_file} ({size_mb:.2f} MB)")
    else:
        print(f"Output file was not created at {output_file}")
        
except Exception as e:
    print(f"Error running traffic monitoring: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# Alternative simplified approach if the above cell still has issues
# This cell provides a simpler way to run the traffic monitoring script
# with less detailed progress reporting but more reliability

import subprocess
import time
import os
import sys
from IPython.display import clear_output
from pathlib import Path

# Only run this cell if the previous cell had issues
print("Only run this cell if the previous approach failed or got stuck.")
print("This is a simplified alternative approach with basic progress reporting.")

# Function to execute command with basic timeout
def run_simple_process(cmd, timeout=1800):
    """Run a process with basic timeout and minimal output handling"""
    print(f"Running command with {timeout}s timeout: {' '.join(cmd)}")
    
    try:
        # Run process with timeout
        process = subprocess.Popen(
            cmd,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            universal_newlines=True
        )
        
        # Start time for periodic updates
        start_time = time.time()
        last_update = start_time
        
        # Periodically check status without relying on stdout reading
        while process.poll() is None:
            current_time = time.time()
            elapsed = current_time - start_time
            
            # Show simple progress update every 5 seconds
            if current_time - last_update >= 5:
                clear_output(wait=True)
                print(f"Process running... Elapsed time: {int(elapsed)}s")
                print(f"If no updates appear for more than 2 minutes, the process may be stuck.")
                last_update = current_time
            
            # Check for timeout
            if elapsed > timeout:
                print(f"Process exceeded timeout of {timeout}s. Terminating...")
                process.terminate()
                try:
                    process.wait(timeout=5)
                except subprocess.TimeoutExpired:
                    process.kill()
                return 124
            
            # Sleep briefly to prevent CPU thrashing
            time.sleep(1)
        
        # Process has completed
        return_code = process.returncode
        print(f"Process completed with return code: {return_code}")
        return return_code
        
    except KeyboardInterrupt:
        print("Process interrupted by user")
        if process and process.poll() is None:
            process.terminate()
            try:
                process.wait(timeout=5)
            except subprocess.TimeoutExpired:
                process.kill()
        return 1
    except Exception as e:
        print(f"Error: {e}")
        return 1

# Get the command arguments (reuse from previous cell)
if 'cmd' not in locals():
    # We need to set up the command
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    output_dir = Path('data/recordings')
    output_dir.mkdir(parents=True, exist_ok=True)
    output_file = str(output_dir / f"traffic_monitoring_simple_{timestamp}.mp4")
    
    # Get video source from config
    with open(config_path, 'r') as file:
        config = yaml.safe_load(file)
        video_source = config['video']['source']
    
    if IN_COLAB or IN_KAGGLE:
        cmd = [sys.executable, 'main.py', '--no-ui', '--record', f'--output={output_file}', f'--source={video_source}']
    else:
        cmd = [sys.executable, 'main.py', '--record', f'--output={output_file}', f'--source={video_source}']

print(f"Command will use video source: {video_source}")
print(f"Output will be saved to: {output_file}")

# Only execute if user confirms
user_input = input("Type 'run' to execute this alternative approach (or press Enter to skip): ")
if user_input.lower() == 'run':
    return_code = run_simple_process(cmd)
    if return_code == 0:
        print("\n✅ Process completed successfully!")
    else:
        print(f"\n⚠️ Process completed with non-zero return code: {return_code}")
    
    # Check output file
    if os.path.exists(output_file):
        size_mb = os.path.getsize(output_file) / (1024 * 1024)
        print(f"Output file created: {output_file} ({size_mb:.2f} MB)")
    else:
        print(f"Output file was not created at {output_file}")
else:
    print("Alternative approach skipped.")


In [ ]:
# Alternative simplified approach if the above cell still has issues
# This cell provides a simpler way to run the traffic monitoring script
# with less detailed progress reporting but more reliability

import subprocess
import time
import os
import sys
from IPython.display import clear_output
from pathlib import Path

# Only run this cell if the previous cell had issues
print("Only run this cell if the previous approach failed or got stuck.")
print("This is a simplified alternative approach with basic progress reporting.")

# Function to execute command with basic timeout
def run_simple_process(cmd, timeout=1800):
    """Run a process with basic timeout and minimal output handling"""
    print(f"Running command with {timeout}s timeout: {' '.join(cmd)}")
    
    try:
        # Run process with timeout
        process = subprocess.Popen(
            cmd,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            universal_newlines=True
        )
        
        # Start time for periodic updates
        start_time = time.time()
        last_update = start_time
        
        # Periodically check status without relying on stdout reading
        while process.poll() is None:
            current_time = time.time()
            elapsed = current_time - start_time
            
            # Show simple progress update every 5 seconds
            if current_time - last_update >= 5:
                clear_output(wait=True)
                print(f"Process running... Elapsed time: {int(elapsed)}s")
                print(f"If no updates appear for more than 2 minutes, the process may be stuck.")
                last_update = current_time
            
            # Check for timeout
            if elapsed > timeout:
                print(f"Process exceeded timeout of {timeout}s. Terminating...")
                process.terminate()
                try:
                    process.wait(timeout=5)
                except subprocess.TimeoutExpired:
                    process.kill()
                return 124
            
            # Sleep briefly to prevent CPU thrashing
            time.sleep(1)
        
        # Process has completed
        return_code = process.returncode
        print(f"Process completed with return code: {return_code}")
        return return_code
        
    except KeyboardInterrupt:
        print("Process interrupted by user")
        if process and process.poll() is None:
            process.terminate()
            try:
                process.wait(timeout=5)
            except subprocess.TimeoutExpired:
                process.kill()
        return 1
    except Exception as e:
        print(f"Error: {e}")
        return 1

# Get the command arguments (reuse from previous cell)
if 'cmd' not in locals():
    # We need to set up the command
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    output_dir = Path('data/recordings')
    output_dir.mkdir(parents=True, exist_ok=True)
    output_file = str(output_dir / f"traffic_monitoring_simple_{timestamp}.mp4")
    
    # Get video source from config
    with open(config_path, 'r') as file:
        config = yaml.safe_load(file)
        video_source = config['video']['source']
    
    if IN_COLAB or IN_KAGGLE:
        cmd = [sys.executable, 'main.py', '--no-ui', '--record', f'--output={output_file}', f'--source={video_source}']
    else:
        cmd = [sys.executable, 'main.py', '--record', f'--output={output_file}', f'--source={video_source}']

print(f"Command will use video source: {video_source}")
print(f"Output will be saved to: {output_file}")

# Only execute if user confirms
user_input = input("Type 'run' to execute this alternative approach (or press Enter to skip): ")
if user_input.lower() == 'run':
    return_code = run_simple_process(cmd)
    if return_code == 0:
        print("\n✅ Process completed successfully!")
    else:
        print(f"\n⚠️ Process completed with non-zero return code: {return_code}")
    
    # Check output file
    if os.path.exists(output_file):
        size_mb = os.path.getsize(output_file) / (1024 * 1024)
        print(f"Output file created: {output_file} ({size_mb:.2f} MB)")
    else:
        print(f"Output file was not created at {output_file}")
else:
    print("Alternative approach skipped.")


## 8. Display Results

Let's display the output video and provide download options.

In [ ]:
from IPython.display import HTML, display
from base64 import b64encode

# Function to create HTML video player for notebook
def display_video(video_path):
    if not os.path.exists(video_path):
        print(f"Video file not found: {video_path}")
        return
    
    # Get video data
    video_data = open(video_path, 'rb').read()
    
    # Encode as base64
    video_base64 = b64encode(video_data).decode('utf-8')
    
    # Create HTML with video player
    html = f'''
    <div style="display: flex; flex-direction: column; align-items: center;">
        <h3>Processed Video: {os.path.basename(video_path)}</h3>
        <video width="640" height="480" controls>
            <source src="data:video/mp4;base64,{video_base64}" type="video/mp4">
            Your browser does not support the video tag.
        </video>
    </div>
    '''
    
    # Display the video
    display(HTML(html))

# Find and display the most recent output video
output_videos = glob.glob('data/recordings/*.mp4')

if output_videos:
    # Sort by modification time to get the most recent
    latest_video = max(output_videos, key=os.path.getmtime)
    print(f"Displaying most recent output video: {latest_video}")
    display_video(latest_video)
    
    # Provide download link for Colab users
    if IN_COLAB:
        from google.colab import files
        print("\nDownload the processed video:")
        files.download(latest_video)
else:
    print("No output videos found. The processing may have failed or been interrupted.")

## 9. Analyze Results from Database

Optionally, let's explore the data collected in the SQLite database.

In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

# Connect to the database
db_path = 'data/traffic_data.db'

if os.path.exists(db_path):
    try:
        # Connect to the database
        conn = sqlite3.connect(db_path)
        
        # Get table names
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        
        print("Database tables:")
        for table in tables:
            print(f"- {table[0]}")
            
        # Query vehicle counts
        df_counts = pd.read_sql("SELECT * FROM vehicle_counts ORDER BY timestamp DESC LIMIT 100", conn)
        if not df_counts.empty:
            print(f"\nVehicle counts (most recent {len(df_counts)} records):")
            display(df_counts.head())
            
            # Plot counts over time if data exists
            if len(df_counts) > 1:
                plt.figure(figsize=(10, 6))
                plt.plot(df_counts['timestamp'], df_counts['count'], marker='o')
                plt.title('Vehicle Counts Over Time')
                plt.xlabel('Timestamp')
                plt.ylabel('Count')
                plt.xticks(rotation=45)
                plt.tight_layout()
                plt.show()
        
        # Query license plates
        df_plates = pd.read_sql("SELECT * FROM license_plates ORDER BY timestamp DESC LIMIT 100", conn)
        if not df_plates.empty:
            print(f"\nDetected license plates (most recent {len(df_plates)} records):")
            display(df_plates.head())
            
        # Close connection
        conn.close()
        
    except Exception as e:
        print(f"Error accessing database: {e}")
else:
    print(f"Database file not found: {db_path}")

## 10. Conclusion

You've successfully run the Traffic Monitoring System! Here's what we accomplished:

1. Set up the necessary environment
2. Downloaded the required AI models
3. Configured the system by directly updating the config.yaml file
4. Processed a video to detect, track, count vehicles and read license plates
5. Saved and analyzed the results

This approach is more efficient as it leverages the existing main.py infrastructure rather than reimplementing functionality in the notebook.

### Next Steps

- Try different videos and configurations
- Explore the database in more detail for advanced analytics
- If you're interested in the technical details, explore the source code in the repository

## Important GPU Settings

Let's ensure we're using GPU acceleration for better performance.

In [ ]:
# Enable GPU usage for better performance
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

# Check if we have GPU available
gpu_available = False
try:
    import torch
    gpu_available = torch.cuda.is_available()
    if gpu_available:
        gpu_name = torch.cuda.get_device_name(0)
        print(f"✅ GPU detected: {gpu_name}")
        
        # Print CUDA version information
        cuda_version = torch.version.cuda
        print(f"CUDA Version: {cuda_version}")
        
        # Check if GPU has enough memory
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)  # Convert to GB
        print(f"GPU Memory: {gpu_memory:.2f} GB")
        
        # Test tensor operation on GPU to ensure it's working properly
        try:
            test_tensor = torch.ones(1).cuda()
            print("✅ GPU tensor operation successful")
        except Exception as e:
            print(f"⚠️ GPU tensor operation failed: {e}")
            print("Falling back to CPU for safety")
            gpu_available = False
    else:
        print("❌ No CUDA-capable GPU detected with PyTorch")
except ImportError:
    print("PyTorch not installed, checking for GPU with system commands")
    if IN_COLAB:
        # For Google Colab
        try:
            gpu_info = !nvidia-smi -L
            if gpu_info and "GPU" in gpu_info[0]:
                gpu_available = True
                print(f"✅ GPU detected: {gpu_info[0]}")
                # Get GPU memory information
                gpu_mem = !nvidia-smi --query-gpu=memory.total --format=csv,noheader,nounits
                if gpu_mem:
                    print(f"GPU Memory: {int(gpu_mem[0])/1024:.2f} GB")
            else:
                print("❌ No GPU detected with nvidia-smi")
        except:
            pass

# Update configuration to use GPU if available
if 'hardware' not in config:
    config['hardware'] = {}

if 'ocr' not in config:
    config['ocr'] = {}

if gpu_available:
    # Set GPU settings for main processing
    config['hardware']['use_gpu'] = True
    config['hardware']['provider'] = 'cuda'
    config['hardware']['precision'] = 'fp16'  # Using half precision for better performance
    
    # Explicitly enable GPU for OCR
    config['ocr']['use_gpu'] = True
    config['ocr']['gpu_id'] = 0  # Use first GPU
    
    print("✅ Configuration updated to use GPU acceleration for detection and OCR")
    print("Note: EasyOCR will use CUDA for license plate recognition")
    
    # Create a warning if we're in Colab but using CPU for OCR
    if IN_COLAB and not config['ocr']['use_gpu']:
        print("⚠️ Warning: Running in Colab with GPU available but OCR was set to use CPU")
        print("   This has been fixed - OCR will now use GPU acceleration")
else:
    # Fallback to CPU
    config['hardware']['use_gpu'] = False
    config['hardware']['provider'] = 'cpu'
    config['hardware']['precision'] = 'fp32'
    
    config['ocr']['use_gpu'] = False
    if 'gpu_id' in config['ocr']:
        del config['ocr']['gpu_id']  # Remove GPU ID if present
    
    print("⚠️ No GPU detected, using CPU mode for all operations (will be slower)")

# Save the updated config
with open(config_path, 'w') as file:
    yaml.dump(config, file, default_flow_style=False)

print(f"\nHardware configuration: {config['hardware']}")
print(f"OCR configuration: {config['ocr']}")

# If GPU is available, add a specific check for EasyOCR
if gpu_available:
    try:
        import easyocr
        # Initialize reader with GPU flag explicitly set
        print("\nTesting EasyOCR GPU support...")
        reader = easyocr.Reader(['en'], gpu=True, verbose=False)
        print("✅ EasyOCR initialized with GPU support successfully")
    except Exception as e:
        print(f"⚠️ Error testing EasyOCR GPU support: {e}")
        print("EasyOCR may still work in the main application with proper configuration")

## Fix MQTT Connection Issues

Let's solve the 'Cannot assign requested address' MQTT error by setting up a proper MQTT configuration or disabling it if not needed.

In [ ]:
# Fix MQTT connection issues
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

# In notebook environments, we'll disable MQTT by default to prevent connection errors
if 'mqtt' not in config:
    config['mqtt'] = {}

# Set up MQTT configuration to prevent the "Cannot assign requested address" error
if IN_COLAB or IN_KAGGLE or IN_NOTEBOOK:
    # Option 1: Disable MQTT completely for notebook environments
    config['mqtt']['enabled'] = False
    print("ℹ️ MQTT disabled in notebook environment to prevent connection errors")
else:
    # Option 2: For local environments, use 127.0.0.1 instead of localhost
    config['mqtt']['enabled'] = True
    config['mqtt']['broker'] = '127.0.0.1'
    config['mqtt']['port'] = 1883
    print("✅ MQTT configured to use 127.0.0.1 instead of localhost")

# Save the updated config
with open(config_path, 'w') as file:
    yaml.dump(config, file, default_flow_style=False)

print(f"MQTT configuration: {config['mqtt']}")

## Run Traffic Monitoring with Improved Progress Reporting

We've updated the configuration to use GPU acceleration when available and fixed MQTT connection issues. Now let's run the traffic monitoring system with improved progress reporting to prevent the notebook from appearing stuck.

In [ ]:
import subprocess
import time
import os
import sys
from IPython.display import clear_output, display, HTML
from pathlib import Path

# Create a directory for recordings if it doesn't exist
output_dir = Path('data/recordings')
output_dir.mkdir(parents=True, exist_ok=True)

# Generate timestamp for output file
timestamp = time.strftime("%Y%m%d_%H%M%S")
output_file = str(output_dir / f"traffic_monitoring_{timestamp}.mp4")

print(f"Will save output to: {output_file}")

# Verify we're using a video file, not a camera
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)
    video_source = config['video']['source']

# Check if we're using a camera (which will cause issues)
if video_source == 0 or video_source == '0':
    print("⚠️ ERROR: Cannot use camera source in notebook environment")
    
    # Find available videos
    available_videos = []
    for ext in ['*.mp4', '*.avi', '*.MOV', '*.mkv']:
        available_videos.extend(glob.glob(os.path.join('data', ext)))
    
    if available_videos:
        video_source = available_videos[0]
        config['video']['source'] = video_source
        with open(config_path, 'w') as file:
            yaml.dump(config, file, default_flow_style=False)
        print(f"✅ Automatically switched to video file: {video_source}")
    else:
        print("❌ No video files found. Please upload a video file.")
        raise ValueError("Cannot proceed without a valid video source")

print(f"Using video source: {video_source}")

# Create a visual status indicator
def create_status_indicator(status, message):
    color = "green" if status == "running" else "red" if status == "error" else "blue"
    return HTML(f"""
    <div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px; margin: 10px 0;">
        <div style="display: flex; align-items: center;">
            <div style="width: 15px; height: 15px; border-radius: 50%; background-color: {color}; margin-right: 10px;"></div>
            <div><b>Status:</b> {message}</div>
        </div>
    </div>
    """)

# Display initial status
status_display = display(create_status_indicator("initializing", "Preparing to run traffic monitoring..."), display_id=True)

# Command to run
if IN_COLAB or IN_KAGGLE:
    # For Colab/Kaggle, use headless mode
    cmd = [sys.executable, 'main.py', '--no-ui', '--record', f'--output={output_file}', f'--source={video_source}']
else:
    # For local notebooks
    cmd = [sys.executable, 'main.py', '--record', f'--output={output_file}', f'--source={video_source}']

print(f"Running command: {' '.join(cmd)}")

# Function to run the command with improved progress reporting
def run_process_with_visual_feedback(cmd):
    process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        universal_newlines=True,
        bufsize=1
    )
    
    status_display.update(create_status_indicator("running", "Traffic monitoring is running..."))
    
    # Variables for progress tracking
    start_time = time.time()
    last_update = time.time()
    update_interval = 2.0  # Update every 2 seconds
    log_buffer = []
    progress_msg = ""
    processed_frames = 0
    total_frames = 0
    
    # Create a progress display
    progress_display = display(HTML(""), display_id=True)
    log_display = display(HTML(""), display_id=True)
    
    try:
        for line in process.stdout:
            # Add line to buffer
            log_buffer.append(line.strip())
            if len(log_buffer) > 20:
                log_buffer = log_buffer[-20:]
            
            # Check for frame processing indicators
            if "Processing frame" in line:
                try:
                    frame_info = line.split("Processing frame")[1].strip()
                    current, total = frame_info.split("/")
                    processed_frames = int(current.strip())
                    total_frames = int(total.strip().split()[0])
                    progress_msg = f"Processing frame {processed_frames}/{total_frames}"
                except:
                    pass
            
            # Check for common error messages
            if "Error" in line or "ERROR" in line or "Failed" in line:
                if "MQTT" not in line:  # Ignore MQTT errors as we've addressed them
                    status_display.update(create_status_indicator("error", f"Error detected: {line.strip()}"))
            
            # Update the display at regular intervals
            current_time = time.time()
            if current_time - last_update > update_interval:
                last_update = current_time
                
                # Calculate progress percentage
                if total_frames > 0:
                    percent = (processed_frames / total_frames) * 100
                    elapsed = current_time - start_time
                    
                    # Estimate time remaining if we have processed at least 10% of frames
                    if percent > 10:
                        eta = (elapsed / processed_frames) * (total_frames - processed_frames)
                        eta_str = f"{eta:.1f} seconds"
                    else:
                        eta_str = "calculating..."
                    
                    # Create a progress bar
                    progress_bar = f"""
                    <div style="width:100%; background-color:#f1f1f1; border-radius:5px;">
                        <div style="width:{percent:.1f}%; height:20px; background-color:#4CAF50; border-radius:5px; text-align:center; color:white;">
                            {percent:.1f}%
                        </div>
                    </div>
                    <div style="margin-top:5px;">
                        <b>Progress:</b> {processed_frames}/{total_frames} frames | <b>Elapsed:</b> {elapsed:.1f}s | <b>ETA:</b> {eta_str}
                    </div>
                    """
                    progress_display.update(HTML(progress_bar))
                
                # Update log display
                log_html = "<div style='font-family:monospace; max-height:200px; overflow:auto; white-space:pre-wrap;'>"
                log_html += "<br>".join(log_buffer[-10:])  # Show last 10 lines
                log_html += "</div>"
                log_display.update(HTML(log_html))
    
    except KeyboardInterrupt:
        status_display.update(create_status_indicator("error", "Process interrupted by user"))
        process.terminate()
        return 1
    
    # Wait for process to complete
    return_code = process.wait()
    
    if return_code == 0:
        status_display.update(create_status_indicator("running", "Processing completed successfully!"))
    else:
        status_display.update(create_status_indicator("error", f"Process failed with return code {return_code}"))
    
    return return_code

# Run the process with visual feedback
try:
    return_code = run_process_with_visual_feedback(cmd)
    if return_code == 0:
        print(f"\n✅ Processing completed successfully!")
        
        # Show the output file path
        if os.path.exists(output_file):
            print(f"Output saved to: {output_file}")
            output_size_mb = os.path.getsize(output_file) / (1024 * 1024)
            print(f"Output file size: {output_size_mb:.2f} MB")
    else:
        print(f"\n❌ Process failed with return code: {return_code}")
except Exception as e:
    print(f"\n❌ Error running traffic monitoring: {e}")